## Pandas analysis

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a couple of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [1]:
# If you didn't download it yet, please get the relevant file now!
!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ~/data/

--2021-11-29 13:43:29--  https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.69.18
Connecting to www.dropbox.com (www.dropbox.com)|162.125.69.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/xvjzaxzz3ysphme/data_000637.txt [following]
--2021-11-29 13:43:29--  https://www.dropbox.com/s/raw/xvjzaxzz3ysphme/data_000637.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc6caac1427aa673ad4e3fa5e041.dl.dropboxusercontent.com/cd/0/inline/Ba68OQciWIMlKjd9BEWSrhG0DIog4brvqn4uep0voyROzJVvxxDwis8iFDLx5KgeSj6rZfbMCbXI2jt-T8w07_eHpBy5jNlYbEpxMtxYHXGUxt3NbcfpB6Z9_Xe_8obxohgy5ichvyZA8v8rQmZe4cdb/file# [following]
--2021-11-29 13:43:29--  https://uc6caac1427aa673ad4e3fa5e041.dl.dropboxusercontent.com/cd/0/inline/Ba68OQciWIMlKjd9BEWSrhG0DIog4brvqn4uep0voyROzJVvxxDwis8iFDLx5KgeSj6rZfbMCbXI2jt-T8w07_eHpBy5jNlYbEpxMtxYHXG

  7600K .......... .......... .......... .......... .......... 23% 77.5M 3s
  7650K .......... .......... .......... .......... .......... 23% 12.7M 3s
  7700K .......... .......... .......... .......... .......... 23%  109M 3s
  7750K .......... .......... .......... .......... .......... 24% 21.2M 3s
  7800K .......... .......... .......... .......... .......... 24% 60.3M 3s
  7850K .......... .......... .......... .......... .......... 24%  113M 3s
  7900K .......... .......... .......... .......... .......... 24% 46.3M 3s
  7950K .......... .......... .......... .......... .......... 24% 22.9M 3s
  8000K .......... .......... .......... .......... .......... 24% 22.9M 3s
  8050K .......... .......... .......... .......... .......... 24% 17.1M 3s
  8100K .......... .......... .......... .......... .......... 25% 25.9M 3s
  8150K .......... .......... .......... .......... .......... 25% 15.6M 3s
  8200K .......... .......... .......... .......... .......... 25% 12.6M 3s
  8250K ....

1\. Create a Pandas DataFrame reading N rows of the 'data_000637.txt' dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k.

In [25]:
import pandas as pd
import random

file_name = "data_000637.txt"
#1310719 highest row index
N = random.randint(10000, 1310719) 
df=pd.read_csv(file_name, nrows = N)
df

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
1268823,1,1,8,3869210887,36,14
1268824,1,0,62,3869210887,35,2
1268825,1,1,13,3869210887,41,5
1268826,1,0,50,3869210887,41,0


2\. Find out the number of BX in a ORBIT (the value 'x').

In [28]:
#Each TDC count corresponds to 25/30 ns,
#whereas a unit of BX_COUNTER corresponds to 25 ns, 
#and the ORBIT_CNT is increased every 'x' BX_COUNTER

x = df['BX_COUNTER'].max()+1
print(x)

3564


In [ ]:
3\. Find out how much the data taking lasted. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading out the whole dataset.

In [ ]:
4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information).

In [ ]:
5\. Replace the values (all 1) of the HEAD column randomly with 0 or 1.

In [ ]:
6\. Create a new DataFrame that contains only the rows with HEAD=1.

In [ ]:
7\. Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel

In [ ]:
8\. Use the groupby method to find out the noisy channels, i.e. the TDC channels with most counts (say the top 3)

In [ ]:
9\. Count the number of unique orbits. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139